In [1]:
import numpy as np
import pandas as pd

/var/folders/n5/f6xjtlr526zbr86__yns243m0000gn/T/ipykernel_10246/1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
bx_cleaned = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/bx_cleaned')

In [3]:
gbx_cleaned = pd.read_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/gbx_cleaned')

In [4]:
import nltk #using the nltk library for tokenization and lemmatization
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sanjeevani1109/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [6]:
#Function to clean text data
def clean_text(text):
    text = text.lower()     #Converting the text to lowercase
    text = re.sub(r'\d+', ' ', text)    # Removing digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Removing special characters
    text = text.translate(str.maketrans('', '', string.punctuation))    #Removing punctuation     
    tokens = word_tokenize(text) #Tokenization of the text
    non_stopwords = []  #Creating an empty list to store the non-stopwords
    stop_words = set(stopwords.words('english')) #Initializing the NLTK English stopwords
    #Lemmatization of the words/tokens
    lemmatizer = WordNetLemmatizer()    #Initialization of the NLTK WordNet Lemmatizer

    #Iterating through the list of tokens, lemmatizing them, and adding it to the list of non-stopwords
    for token in tokens:
        if token not in stop_words:
            lemma = lemmatizer.lemmatize(token) #Performing lemmatization
            non_stopwords.append(lemma)  # Appending the stemmed token back to the list of non-stopwords

    # Joining the tokens/words from the list of non-stopwords into a string
    filtered_text = ' '.join(non_stopwords)
    return filtered_text

In [7]:
def clean_genres(text):
    text = text.lower()
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [8]:
pip install transformers

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install torch torchvision torchaudio

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [10]:
from transformers import BertTokenizer, BertModel

In [11]:
# Initializing BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [12]:
# Generate BERT embeddings for a list of texts in batches

def get_bert_embeddings_batch(text_list, batch_size=32):
    embeddings = [] #initializign an empty list to store the BERT embeddings for all the texts
    for i in range(0, len(text_list), batch_size):  #batch processing the texts from the list of texts
        batch = text_list[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512) #tokenization of the texts in the batch and returning the PyTorch tensors
        outputs = model(**inputs)   #passing the tokenized inputs to the BERT model to retrieve the outputs
        embeddings.extend(outputs.last_hidden_state.mean(dim=1).detach().numpy()) 
    return np.vstack(embeddings)    #vertical stacking of of the embeddings to create a single matrix of embeddings for the input texts

In [15]:
bx_cleaned

,UserID,ISBN,User_Rating,Title,Author,Average_Rating,cleaned_title,cleaned_author,Genres,new_combined,title_embeddings,combined_embeddings,genres_embeddings,combined_cluster_label,genre_cluster_label
0,276733,2080674722,1,Les Particules Elementaires,Michel Houellebecq,2,le particules elementaires,michel houellebecq,"Fiction, France, Literature, Novels, French Li...",le particules elementaires michel houellebecq ...,[-2.23144725e-01 4.20064986e-01 -2.87288487e-...,[-2.10876107e-01 3.81659061e-01 -6.62146893e-...,[ 1.18415318e-01 2.04688787e-01 -2.81599686e-...,1,3
1,276746,0425115801,1,Lightning,Dean R. Koontz,2,lightning,dean r koontz,"Fantasy, Young Adult, Mythology, Fiction, Midd...",lightning dean r koontz fantasy young adult my...,[-2.50022531e-01 1.23583212e-01 9.59393233e-...,[-1.25961661e-01 2.70925760e-01 1.89602107e-...,[ 5.65221347e-02 2.65754551e-01 1.27978697e-...,3,4
2,276746,0449006522,1,Manhattan Hunt Club,JOHN SAUL,2,manhattan hunt club,john saul,"Horror, Fiction, Thriller, Mystery, Suspense, ...",manhattan hunt club john saul horror fiction t...,[-6.91263303e-02 -1.97221622e-01 -3.38341296e-...,[ 3.31362695e-01 -1.22959260e-02 1.94353983e-...,[ 3.01900238e-01 1.63262621e-01 5.86322367e-...,2,1
3,276746,0553561618,1,Dark Paradise,TAMI HOAG,2,dark paradise,tami hoag,"Mystery, Romance, Suspense, Fiction, Romantic ...",dark paradise tami hoag mystery romance suspen...,[-1.85212106e-01 3.52137908e-02 -4.30807360e-...,[ 1.47180423e-01 4.18665679e-03 1.40036240e-...,[ 3.27225596e-01 1.61523923e-01 5.92916429e-...,2,1
4,276747,0451192001,1,How Stella Got Her Groove Back,Terry McMillan,1,stella got groove back,terry mcmillan,"Fiction, Romance, African American, Chick Lit,...",stella got groove back terry mcmillan fiction ...,[ 2.10772641e-02 -3.62699986e-01 5.60501888e-...,[-1.07199050e-01 -1.08777201e-02 2.86322713e-...,[-6.88135326e-02 7.82288145e-03 1.24400854e-...,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336313,276704,0441007813,1,Obsidian Butterfly,Laurell K. Hamilton,2,obsidian butterfly,laurell k hamilton,"Urban Fantasy, Vampires, Paranormal, Fantasy, ...",obsidian butterfly laurell k hamilton urban fa...,[-2.61512458e-01 -8.30207467e-02 -2.90368706e-...,[ 2.03595653e-01 1.13599174e-01 4.86624211e-...,[ 4.00419414e-01 2.76881635e-01 5.94798684e-...,2,1
336314,276704,0446353957,1,Mirror Image,Sandra Brown,2,mirror image,sandra brown,"Romance, Romantic Suspense, Mystery, Suspense,...",mirror image sandra brown romance romantic sus...,[-1.37333304e-01 -8.67070183e-02 -9.49039299e-...,[ 1.09254815e-01 -1.22819684e-01 9.29117873e-...,[ 2.13263214e-01 1.43371105e-01 4.22883481e-...,2,1
336315,276704,0743211383,4,Dreamcatcher,Stephen King,2,dreamcatcher,stephen king,"Horror, Fiction, Science Fiction, Thriller, Fa...",dreamcatcher stephen king horror fiction scien...,[-1.27313450e-01 -5.03183961e-01 1.43543914e-...,[ 1.60899252e-01 1.77120626e-01 2.89601404e-...,[ 3.28216821e-01 3.93379867e-01 2.72073925e-...,2,2
336316,276704,080410526X,1,All I Really Need to Know,ROBERT FULGHUM,2,really need know,robert fulghum,"Nonfiction, Humor, Self Help, Philosophy, Essa...",really need know robert fulghum nonfiction hum...,[-1.94892079e-01 1.58075050e-01 1.58971533e-...,[ 1.37256518e-01 2.73359865e-01 8.69663134e-...,[-8.39283573e-04 3.47034216e-01 -4.28967588e-...,0,3


In [16]:
gbx_cleaned

,id,BookID,ISBN,Author,Title,Average_Rating,UserID,User_Rating,cleaned_title,cleaned_author,Genres,new_combined,title_embeddings,combined_embeddings,genres_embeddings,combined_cluster_label,genre_cluster_label
0,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,314,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,[-3.51202160e-01 -6.33739769e-01 -2.95057833e-...,[-1.05580039e-01 6.42038509e-02 4.10709947e-...,[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
1,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,439,3,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,[-3.51202160e-01 -6.33739769e-01 -2.95057833e-...,[-1.05580039e-01 6.42038509e-02 4.10709947e-...,[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
2,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,588,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,[-3.51202160e-01 -6.33739769e-01 -2.95057833e-...,[-1.05580039e-01 6.42038509e-02 4.10709947e-...,[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
3,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,1169,4,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,[-3.51202160e-01 -6.33739769e-01 -2.95057833e-...,[-1.05580039e-01 6.42038509e-02 4.10709947e-...,[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
4,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,1185,4,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,[-3.51202160e-01 -6.33739769e-01 -2.95057833e-...,[-1.05580039e-01 6.42038509e-02 4.10709947e-...,[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819645,10000,8914,0375400524,John Keegan,The First World War,4,48281,4,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,[-6.09237477e-02 -2.36533821e-01 -1.66040897e-...,[ 1.98690891e-01 5.79691708e-01 -3.13248068e-...,[ 1.47486791e-01 3.88453394e-01 -8.60980973e-...,2,2
819646,10000,8914,0375400524,John Keegan,The First World War,4,48386,5,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,[-6.09237477e-02 -2.36533821e-01 -1.66040897e-...,[ 1.98690891e-01 5.79691708e-01 -3.13248068e-...,[ 1.47486791e-01 3.88453394e-01 -8.60980973e-...,2,2
819647,10000,8914,0375400524,John Keegan,The First World War,4,49007,4,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,[-6.09237477e-02 -2.36533821e-01 -1.66040897e-...,[ 1.98690891e-01 5.79691708e-01 -3.13248068e-...,[ 1.47486791e-01 3.88453394e-01 -8.60980973e-...,2,2
819648,10000,8914,0375400524,John Keegan,The First World War,4,49383,5,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,[-6.09237477e-02 -2.36533821e-01 -1.66040897e-...,[ 1.98690891e-01 5.79691708e-01 -3.13248068e-...,[ 1.47486791e-01 3.88453394e-01 -8.60980973e-...,2,2


### Generating BERT embeddings for 'Title' columns of gbx_cleaned and bx_cleaned data frames

In [17]:
unique_titles_gbx = gbx_cleaned['Title'].drop_duplicates().reset_index(drop=True)
gbx_title_embedding = get_bert_embeddings_batch(unique_titles_gbx.tolist())

#Mapping Titles to the Embeddings
title_to_embedding_gbx = dict(zip(unique_titles_gbx, gbx_title_embedding))

#Mapping embeddings to the original data frame
gbx_cleaned['title_embeddings'] = gbx_cleaned['Title'].map(title_to_embedding_gbx)

gbx_cleaned

,id,BookID,ISBN,Author,Title,Average_Rating,UserID,User_Rating,cleaned_title,cleaned_author,Genres,new_combined,title_embeddings,combined_embeddings,genres_embeddings,combined_cluster_label,genre_cluster_label
0,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,314,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...",[-1.05580039e-01 6.42038509e-02 4.10709947e-...,[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
1,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,439,3,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...",[-1.05580039e-01 6.42038509e-02 4.10709947e-...,[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
2,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,588,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...",[-1.05580039e-01 6.42038509e-02 4.10709947e-...,[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
3,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,1169,4,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...",[-1.05580039e-01 6.42038509e-02 4.10709947e-...,[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
4,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,1185,4,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...",[-1.05580039e-01 6.42038509e-02 4.10709947e-...,[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819645,10000,8914,0375400524,John Keegan,The First World War,4,48281,4,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,"[-0.060923338, -0.23653369, -0.16604085, -0.21...",[ 1.98690891e-01 5.79691708e-01 -3.13248068e-...,[ 1.47486791e-01 3.88453394e-01 -8.60980973e-...,2,2
819646,10000,8914,0375400524,John Keegan,The First World War,4,48386,5,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,"[-0.060923338, -0.23653369, -0.16604085, -0.21...",[ 1.98690891e-01 5.79691708e-01 -3.13248068e-...,[ 1.47486791e-01 3.88453394e-01 -8.60980973e-...,2,2
819647,10000,8914,0375400524,John Keegan,The First World War,4,49007,4,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,"[-0.060923338, -0.23653369, -0.16604085, -0.21...",[ 1.98690891e-01 5.79691708e-01 -3.13248068e-...,[ 1.47486791e-01 3.88453394e-01 -8.60980973e-...,2,2
819648,10000,8914,0375400524,John Keegan,The First World War,4,49383,5,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,"[-0.060923338, -0.23653369, -0.16604085, -0.21...",[ 1.98690891e-01 5.79691708e-01 -3.13248068e-...,[ 1.47486791e-01 3.88453394e-01 -8.60980973e-...,2,2


In [18]:
unique_titles_bx = bx_cleaned['Title'].drop_duplicates().reset_index(drop=True)
bx_title_embedding = get_bert_embeddings_batch(unique_titles_bx.tolist())
title_to_embedding_bx = dict(zip(unique_titles_bx, bx_title_embedding))

#Mapping embeddings to the original data frame
bx_cleaned['title_embeddings'] = bx_cleaned['Title'].map(title_to_embedding_bx)

bx_cleaned

### Generating BERT embeddings for 'new_combined' columns of gbx_cleaned and bx_cleaned data frames

In [ ]:
unique_combined_gbx = gbx_cleaned['new_combined'].drop_duplicates().reset_index(drop=True)
gbx_combined_embedding = get_bert_embeddings_batch(unique_combined_gbx.tolist())
combined_to_embedding_gbx = dict(zip(unique_combined_gbx, gbx_combined_embedding))
gbx_cleaned['combined_embeddings'] = gbx_cleaned['new_combined'].map(combined_to_embedding_gbx)

gbx_cleaned

,id,BookID,ISBN,Author,Title,Average_Rating,UserID,User_Rating,cleaned_title,cleaned_author,Genres,new_combined,title_embeddings,combined_embeddings,genres_embeddings,combined_cluster_label,genre_cluster_label
0,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,314,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...",[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
1,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,439,3,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...",[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
2,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,588,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...",[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
3,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,1169,4,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...",[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
4,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,1185,4,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...",[-3.41960602e-02 1.48707554e-01 5.22670746e-...,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819645,10000,8914,0375400524,John Keegan,The First World War,4,48281,4,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,"[-0.060923338, -0.23653369, -0.16604085, -0.21...","[0.19869067, 0.5796916, -0.31324795, -0.231550...",[ 1.47486791e-01 3.88453394e-01 -8.60980973e-...,2,2
819646,10000,8914,0375400524,John Keegan,The First World War,4,48386,5,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,"[-0.060923338, -0.23653369, -0.16604085, -0.21...","[0.19869067, 0.5796916, -0.31324795, -0.231550...",[ 1.47486791e-01 3.88453394e-01 -8.60980973e-...,2,2
819647,10000,8914,0375400524,John Keegan,The First World War,4,49007,4,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,"[-0.060923338, -0.23653369, -0.16604085, -0.21...","[0.19869067, 0.5796916, -0.31324795, -0.231550...",[ 1.47486791e-01 3.88453394e-01 -8.60980973e-...,2,2
819648,10000,8914,0375400524,John Keegan,The First World War,4,49383,5,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,"[-0.060923338, -0.23653369, -0.16604085, -0.21...","[0.19869067, 0.5796916, -0.31324795, -0.231550...",[ 1.47486791e-01 3.88453394e-01 -8.60980973e-...,2,2


In [ ]:
unique_combined_bx = bx_cleaned['new_combined'].drop_duplicates().reset_index(drop=True)
bx_combined_embedding = get_bert_embeddings_batch(unique_combined_bx.tolist())
combined_to_embedding_bx = dict(zip(unique_combined_bx, bx_combined_embedding))
bx_cleaned['combined_embeddings'] = bx_cleaned['new_combined'].map(combined_to_embedding_bx)

bx_cleaned

,UserID,ISBN,User_Rating,Title,Author,Average_Rating,cleaned_title,cleaned_author,Genres,new_combined,title_embeddings,combined_embeddings,genres_embeddings,combined_cluster_label,genre_cluster_label
0,276733,2080674722,1,Les Particules Elementaires,Michel Houellebecq,2,le particules elementaires,michel houellebecq,"Fiction, France, Literature, Novels, French Li...",le particules elementaires michel houellebecq ...,"[-0.22314462, 0.42006478, -0.28728843, -0.1840...","[-0.21087638, 0.38165882, -0.00066206243, -0.3...",[ 1.18415318e-01 2.04688787e-01 -2.81599686e-...,1,3
1,276746,0425115801,1,Lightning,Dean R. Koontz,2,lightning,dean r koontz,"Fantasy, Young Adult, Mythology, Fiction, Midd...",lightning dean r koontz fantasy young adult my...,"[-0.2500226, 0.1235834, 0.09593897, -0.0904388...","[-0.12596153, 0.2709259, 0.18960203, 0.0491416...",[ 5.65221347e-02 2.65754551e-01 1.27978697e-...,3,4
2,276746,0449006522,1,Manhattan Hunt Club,JOHN SAUL,2,manhattan hunt club,john saul,"Horror, Fiction, Thriller, Mystery, Suspense, ...",manhattan hunt club john saul horror fiction t...,"[-0.06912649, -0.19722155, -0.33834106, 0.1077...","[0.33136278, -0.012296129, 0.19435395, 0.10583...",[ 3.01900238e-01 1.63262621e-01 5.86322367e-...,2,1
3,276746,0553561618,1,Dark Paradise,TAMI HOAG,2,dark paradise,tami hoag,"Mystery, Romance, Suspense, Fiction, Romantic ...",dark paradise tami hoag mystery romance suspen...,"[-0.18521202, 0.035213947, -0.043080643, 0.062...","[0.14718063, 0.0041864626, 0.14003608, 0.03475...",[ 3.27225596e-01 1.61523923e-01 5.92916429e-...,2,1
4,276747,0451192001,1,How Stella Got Her Groove Back,Terry McMillan,1,stella got groove back,terry mcmillan,"Fiction, Romance, African American, Chick Lit,...",stella got groove back terry mcmillan fiction ...,"[0.021076923, -0.36269993, 0.056049906, 0.0045...","[-0.10719866, -0.010877345, 0.28632286, 0.0426...",[-6.88135326e-02 7.82288145e-03 1.24400854e-...,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336313,276704,0441007813,1,Obsidian Butterfly,Laurell K. Hamilton,2,obsidian butterfly,laurell k hamilton,"Urban Fantasy, Vampires, Paranormal, Fantasy, ...",obsidian butterfly laurell k hamilton urban fa...,"[-0.26151246, -0.08302074, -0.29036877, -0.032...","[0.20359544, 0.11359893, 0.48662427, 0.0611085...",[ 4.00419414e-01 2.76881635e-01 5.94798684e-...,2,1
336314,276704,0446353957,1,Mirror Image,Sandra Brown,2,mirror image,sandra brown,"Romance, Romantic Suspense, Mystery, Suspense,...",mirror image sandra brown romance romantic sus...,"[-0.13733369, -0.08670683, -0.00094897894, 0.0...","[0.109255105, -0.12281986, 0.09291214, -0.0980...",[ 2.13263214e-01 1.43371105e-01 4.22883481e-...,2,1
336315,276704,0743211383,4,Dreamcatcher,Stephen King,2,dreamcatcher,stephen king,"Horror, Fiction, Science Fiction, Thriller, Fa...",dreamcatcher stephen king horror fiction scien...,"[-0.12731329, -0.50318426, 0.14354372, 0.11408...","[0.16089956, 0.17712127, 0.028960295, 0.070600...",[ 3.28216821e-01 3.93379867e-01 2.72073925e-...,2,2
336316,276704,080410526X,1,All I Really Need to Know,ROBERT FULGHUM,2,really need know,robert fulghum,"Nonfiction, Humor, Self Help, Philosophy, Essa...",really need know robert fulghum nonfiction hum...,"[-0.19489235, 0.15807506, 0.15897146, 0.023295...","[0.13725634, 0.27335998, 0.08696638, -0.101382...",[-8.39283573e-04 3.47034216e-01 -4.28967588e-...,0,3


### Generating BERT embeddings for 'Genres' columns of gbx_cleaned and bx_cleaned data frames

In [ ]:
unique_genres_gbx = gbx_cleaned['Genres'].drop_duplicates().reset_index(drop=True)
gbx_genres_embedding = get_bert_embeddings_batch(unique_genres_gbx.tolist())
genres_to_embedding_gbx = dict(zip(unique_genres_gbx, gbx_genres_embedding))
gbx_cleaned['genres_embeddings'] = gbx_cleaned['Genres'].map(genres_to_embedding_gbx)

gbx_cleaned

,id,BookID,ISBN,Author,Title,Average_Rating,UserID,User_Rating,cleaned_title,cleaned_author,Genres,new_combined,title_embeddings,combined_embeddings,genres_embeddings,combined_cluster_label,genre_cluster_label
0,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,314,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...","[-0.03419616, 0.14870769, 0.5226705, -0.080204...",3,1
1,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,439,3,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...","[-0.03419616, 0.14870769, 0.5226705, -0.080204...",3,1
2,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,588,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...","[-0.03419616, 0.14870769, 0.5226705, -0.080204...",3,1
3,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,1169,4,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...","[-0.03419616, 0.14870769, 0.5226705, -0.080204...",3,1
4,1,2767052,439023483,Suzanne Collins,The Hunger Games,4,1185,4,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...","[-0.03419616, 0.14870769, 0.5226705, -0.080204...",3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819645,10000,8914,0375400524,John Keegan,The First World War,4,48281,4,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,"[-0.060923338, -0.23653369, -0.16604085, -0.21...","[0.19869067, 0.5796916, -0.31324795, -0.231550...","[0.14748658, 0.3884535, -0.08609801, -0.255218...",2,2
819646,10000,8914,0375400524,John Keegan,The First World War,4,48386,5,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,"[-0.060923338, -0.23653369, -0.16604085, -0.21...","[0.19869067, 0.5796916, -0.31324795, -0.231550...","[0.14748658, 0.3884535, -0.08609801, -0.255218...",2,2
819647,10000,8914,0375400524,John Keegan,The First World War,4,49007,4,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,"[-0.060923338, -0.23653369, -0.16604085, -0.21...","[0.19869067, 0.5796916, -0.31324795, -0.231550...","[0.14748658, 0.3884535, -0.08609801, -0.255218...",2,2
819648,10000,8914,0375400524,John Keegan,The First World War,4,49383,5,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...,"[-0.060923338, -0.23653369, -0.16604085, -0.21...","[0.19869067, 0.5796916, -0.31324795, -0.231550...","[0.14748658, 0.3884535, -0.08609801, -0.255218...",2,2


In [ ]:
unique_genres_bx = bx_cleaned['Genres'].drop_duplicates().reset_index(drop=True)
bx_genres_embedding = get_bert_embeddings_batch(unique_genres_bx.tolist())
genres_to_embedding_bx = dict(zip(unique_genres_bx, bx_genres_embedding))
bx_cleaned['genres_embeddings'] = bx_cleaned['Genres'].map(genres_to_embedding_bx)

bx_cleaned

,UserID,ISBN,User_Rating,Title,Author,Average_Rating,cleaned_title,cleaned_author,Genres,new_combined,title_embeddings,combined_embeddings,genres_embeddings,combined_cluster_label,genre_cluster_label
0,276733,2080674722,1,Les Particules Elementaires,Michel Houellebecq,2,le particules elementaires,michel houellebecq,"Fiction, France, Literature, Novels, French Li...",le particules elementaires michel houellebecq ...,"[-0.22314462, 0.42006478, -0.28728843, -0.1840...","[-0.21087638, 0.38165882, -0.00066206243, -0.3...","[0.11841526, 0.20468897, -0.028159915, -0.2183...",1,3
1,276746,0425115801,1,Lightning,Dean R. Koontz,2,lightning,dean r koontz,"Fantasy, Young Adult, Mythology, Fiction, Midd...",lightning dean r koontz fantasy young adult my...,"[-0.2500226, 0.1235834, 0.09593897, -0.0904388...","[-0.12596153, 0.2709259, 0.18960203, 0.0491416...","[0.056521952, 0.26575485, 0.12797885, 0.070500...",3,4
2,276746,0449006522,1,Manhattan Hunt Club,JOHN SAUL,2,manhattan hunt club,john saul,"Horror, Fiction, Thriller, Mystery, Suspense, ...",manhattan hunt club john saul horror fiction t...,"[-0.06912649, -0.19722155, -0.33834106, 0.1077...","[0.33136278, -0.012296129, 0.19435395, 0.10583...","[0.30190024, 0.16326268, 0.5863225, -0.1031865...",2,1
3,276746,0553561618,1,Dark Paradise,TAMI HOAG,2,dark paradise,tami hoag,"Mystery, Romance, Suspense, Fiction, Romantic ...",dark paradise tami hoag mystery romance suspen...,"[-0.18521202, 0.035213947, -0.043080643, 0.062...","[0.14718063, 0.0041864626, 0.14003608, 0.03475...","[0.32722577, 0.1615243, 0.592917, -0.18396236,...",2,1
4,276747,0451192001,1,How Stella Got Her Groove Back,Terry McMillan,1,stella got groove back,terry mcmillan,"Fiction, Romance, African American, Chick Lit,...",stella got groove back terry mcmillan fiction ...,"[0.021076923, -0.36269993, 0.056049906, 0.0045...","[-0.10719866, -0.010877345, 0.28632286, 0.0426...","[-0.06881369, 0.007822593, 0.1244007, -0.25172...",2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336313,276704,0441007813,1,Obsidian Butterfly,Laurell K. Hamilton,2,obsidian butterfly,laurell k hamilton,"Urban Fantasy, Vampires, Paranormal, Fantasy, ...",obsidian butterfly laurell k hamilton urban fa...,"[-0.26151246, -0.08302074, -0.29036877, -0.032...","[0.20359544, 0.11359893, 0.48662427, 0.0611085...","[0.4004193, 0.27688152, 0.59479856, -0.0511764...",2,1
336314,276704,0446353957,1,Mirror Image,Sandra Brown,2,mirror image,sandra brown,"Romance, Romantic Suspense, Mystery, Suspense,...",mirror image sandra brown romance romantic sus...,"[-0.13733369, -0.08670683, -0.00094897894, 0.0...","[0.109255105, -0.12281986, 0.09291214, -0.0980...","[0.21326312, 0.14337139, 0.4228834, -0.1299948...",2,1
336315,276704,0743211383,4,Dreamcatcher,Stephen King,2,dreamcatcher,stephen king,"Horror, Fiction, Science Fiction, Thriller, Fa...",dreamcatcher stephen king horror fiction scien...,"[-0.12731329, -0.50318426, 0.14354372, 0.11408...","[0.16089956, 0.17712127, 0.028960295, 0.070600...","[0.32821694, 0.39338025, 0.27207398, -0.190100...",2,2
336316,276704,080410526X,1,All I Really Need to Know,ROBERT FULGHUM,2,really need know,robert fulghum,"Nonfiction, Humor, Self Help, Philosophy, Essa...",really need know robert fulghum nonfiction hum...,"[-0.19489235, 0.15807506, 0.15897146, 0.023295...","[0.13725634, 0.27335998, 0.08696638, -0.101382...","[-0.0008390044, 0.34703422, -0.04289644, -0.04...",0,3


### Saving the data frames as csv files

In [ ]:
gbx_cleaned.to_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/gbx_final', index=False)

In [ ]:
bx_cleaned.to_csv('/Users/sanjeevani1109/Desktop/Book_Recommendation_System/bx_final', index=False)

# Content-based Model

##### Using BERT embeddings, TFIDF vectorization, and kNN

In [22]:
relevant_columns = ['Title', 'Author', 'Genres', 'cleaned_title', 'cleaned_author', 'new_combined', 'title_embeddings', 'genres_embeddings', 'combined_embeddings']  

In [23]:
# Drop duplicates based on the 'Title' column and select only the relevant columns for bx_cleaned and gbx_cleaned
gbx_content = gbx_cleaned.drop_duplicates(subset='Title')[relevant_columns]
bx_content = bx_cleaned.drop_duplicates(subset='Title')[relevant_columns]

In [24]:
gbx_content

,Title,Author,Genres,cleaned_title,cleaned_author,new_combined,title_embeddings,genres_embeddings,combined_embeddings
0,The Hunger Games,Suzanne Collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game,suzanne collins,hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.03419616, 0.14870769, 0.5226705, -0.080204...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544..."
100,Harry Potter and the Philosopher's Stone,"J.K. Rowling, Mary GrandPrÃ©","Fantasy, Fiction, Young Adult, Magic, Children...",harry potter philosopher stone,jk rowling mary grandpr,harry potter philosopher stone jk rowling mary...,"[-0.0704749, -0.3042533, -0.29448748, -0.05764...","[0.010185583, 0.19501851, 0.5730598, 0.0165502...","[-0.020698978, 0.20193146, 0.5163454, -0.07961..."
200,Twilight,Stephenie Meyer,"Young Adult, Romance, Fiction, Vampires, Paran...",twilight,stephenie meyer,twilight stephenie meyer young adult romance f...,"[0.025059534, -0.024350427, 0.48825073, -0.041...","[0.29063728, 0.18538782, 0.4688043, -0.1585567...","[0.23011772, 0.1269917, 0.34506676, 0.05906544..."
300,To Kill a Mockingbird,Harper Lee,"Classics, Fiction, Historical Fiction, School,...",kill mockingbird,harper lee,kill mockingbird harper lee classics fiction h...,"[0.14425337, -0.05986623, -0.30642003, -0.0321...","[-0.09042584, 0.23808128, 0.40244234, -0.00589...","[-0.0010744382, 0.24940604, 0.2772099, 0.03769..."
400,The Great Gatsby,F. Scott Fitzgerald,"Classics, Fiction, School, Historical Fiction,...",great gatsby,f scott fitzgerald,great gatsby f scott fitzgerald classics ficti...,"[-0.52564627, -0.060652833, 0.19872563, -0.047...","[0.121152796, 0.22836153, 0.32600167, -0.12615...","[0.0018115027, 0.39045668, 0.2720787, 0.01734,..."
...,...,...,...,...,...,...,...,...,...
819174,Bayou Moon,Ilona Andrews,"Urban Fantasy, Fantasy, Romance, Paranormal, P...",bayou moon,ilona andrew,bayou moon ilona andrew urban fantasy fantasy ...,"[0.011776062, 0.018023346, -0.022862965, 0.055...","[0.46652517, 0.25250813, 0.47028634, -0.276397...","[0.17634927, 0.11475245, 0.53516227, -0.094267..."
819272,Means of Ascent,Robert A. Caro,"Biography, History, Politics, Nonfiction, Pres...",mean ascent,robert caro,mean ascent robert caro biography history poli...,"[0.05283801, 0.10213105, -0.3021835, 0.1710091...","[-0.00086233765, 0.4549938, 0.03883616, 0.0463...","[0.14772765, 0.4163406, -0.09833094, -0.017137..."
819361,The Mauritius Command,Patrick O'Brian,"Historical Fiction, Fiction, Historical, Adven...",mauritius command,patrick obrian,mauritius command patrick obrian historical fi...,"[-0.6508945, -0.27501172, -0.27637306, -0.2280...","[0.10557652, 0.18600497, 0.4122532, -0.1863815...","[-0.12552503, 0.28464678, 0.24779034, -0.19777..."
819456,Cinderella Ate My Daughter: Dispatches from th...,Peggy Orenstein,"Nonfiction, Parenting, Feminism, Psychology, S...",cinderella ate daughter dispatch frontlines ne...,peggy orenstein,cinderella ate daughter dispatch frontlines ne...,"[0.0860894, -0.33880308, 0.5204474, -0.0286655...","[0.12289753, 0.2606757, -0.055598345, -0.13644...","[0.030213406, 0.13814645, 0.36173916, -0.03155..."


In [25]:
bx_content

,Title,Author,Genres,cleaned_title,cleaned_author,new_combined,title_embeddings,genres_embeddings,combined_embeddings
0,Les Particules Elementaires,Michel Houellebecq,"Fiction, France, Literature, Novels, French Li...",le particules elementaires,michel houellebecq,le particules elementaires michel houellebecq ...,"[-0.22314462, 0.42006478, -0.28728843, -0.1840...","[0.11841526, 0.20468897, -0.028159915, -0.2183...","[-0.21087638, 0.38165882, -0.00066206243, -0.3..."
1,Lightning,Dean R. Koontz,"Fantasy, Young Adult, Mythology, Fiction, Midd...",lightning,dean r koontz,lightning dean r koontz fantasy young adult my...,"[-0.2500226, 0.1235834, 0.09593897, -0.0904388...","[0.056521952, 0.26575485, 0.12797885, 0.070500...","[-0.12596153, 0.2709259, 0.18960203, 0.0491416..."
2,Manhattan Hunt Club,JOHN SAUL,"Horror, Fiction, Thriller, Mystery, Suspense, ...",manhattan hunt club,john saul,manhattan hunt club john saul horror fiction t...,"[-0.06912649, -0.19722155, -0.33834106, 0.1077...","[0.30190024, 0.16326268, 0.5863225, -0.1031865...","[0.33136278, -0.012296129, 0.19435395, 0.10583..."
3,Dark Paradise,TAMI HOAG,"Mystery, Romance, Suspense, Fiction, Romantic ...",dark paradise,tami hoag,dark paradise tami hoag mystery romance suspen...,"[-0.18521202, 0.035213947, -0.043080643, 0.062...","[0.32722577, 0.1615243, 0.592917, -0.18396236,...","[0.14718063, 0.0041864626, 0.14003608, 0.03475..."
4,How Stella Got Her Groove Back,Terry McMillan,"Fiction, Romance, African American, Chick Lit,...",stella got groove back,terry mcmillan,stella got groove back terry mcmillan fiction ...,"[0.021076923, -0.36269993, 0.056049906, 0.0045...","[-0.06881369, 0.007822593, 0.1244007, -0.25172...","[-0.10719866, -0.010877345, 0.28632286, 0.0426..."
...,...,...,...,...,...,...,...,...,...
161240,Der Rumpf. Roman.,Akif Pirincci,"Horror, Classics, Fiction, Short Stories, Fant...",der rumpf roman,akif pirincci,der rumpf roman akif pirincci horror classics ...,"[-0.21596836, -0.035745386, 0.015203243, -0.23...","[0.24007809, 0.6710358, 0.09675342, -0.2001748...","[0.08018193, 0.5062408, 0.21658997, -0.2734610..."
205109,El Misterio del Cuarto Amarillo,Gaston LeRoux,"Mystery, Classics, Fiction, Crime, France, Mys...",el misterio del cuarto amarillo,gaston leroux,el misterio del cuarto amarillo gaston leroux ...,"[-0.30282417, -0.05225911, 0.06205786, 0.12959...","[0.039093185, 0.14366075, 0.41158238, -0.13086...","[-0.17640495, -0.0092257885, 0.16992046, -0.03..."
221563,Choices: Taking Control of Your Life and Makin...,Melody Beattie,"Self Help, Nonfiction, Personal Development, P...",choice taking control life making matter,melody beattie,choice taking control life making matter melod...,"[0.21352075, -0.11129819, -0.34347287, 0.12421...","[0.22054334, 0.31409207, -0.16236278, -0.00384...","[0.33450943, 0.14688563, -0.03689841, 0.073954..."
275943,African Skies (Palisades Pure Romance),Karen Rispin,"Romance, Christian, Christian Fiction, Adult F...",african sky palisade pure romance,karen rispin,african sky palisade pure romance karen rispin...,"[0.09651271, -0.1985374, -0.20831119, -0.02151...","[0.039885662, 0.18925609, 0.055060387, -0.1999...","[-0.005648445, 0.09101412, 0.067483604, -0.026..."


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import vstack, hstack
import numpy as np

In [27]:
def compute_combined_features(df):
    #Computing TF-IDF Features
    tfidf_vectorizer = TfidfVectorizer(max_features=100)  # Adjust max_features as needed
    tfidf_features = tfidf_vectorizer.fit_transform(df['new_combined'])

    #Concatenating BERT and TF-IDF Features
    bert_features = np.stack(df['combined_embeddings'].to_numpy())  # Use stack for an array of arrays
    combined_features = hstack([bert_features, tfidf_features])

    return combined_features, tfidf_vectorizer

In [28]:
def get_combined_recommendations(query, df, top_k=10):
    # Compute combined features for the specified DataFrame
    combined_features, tfidf_vectorizer = compute_combined_features(df)

    # Use KNN for Recommendations
    knn_model = NearestNeighbors(n_neighbors=top_k, metric='cosine')
    knn_model.fit(combined_features)

    query_embedding = get_bert_embeddings_batch([query])[0].astype(np.float64)
    query_tfidf = tfidf_vectorizer.transform([query])
    query_combined = hstack([np.array([query_embedding]), query_tfidf])

    distances, indices = knn_model.kneighbors(query_combined, n_neighbors=top_k*10)

    # Flatten indices and fetch recommendations
    recommendations = df.iloc[indices.flatten()]

    # Ensure unique recommendations based on Title and Author, excluding the query itself
    unique_titles = set()
    unique_recommendations = []
    for _, row in recommendations.iterrows():
        unique_key = (row['Title'].lower(), row['Author'].lower())
        if unique_key not in unique_titles and row['Title'].lower() != query.lower():
            unique_titles.add(unique_key)
            unique_recommendations.append(row)
            if len(unique_recommendations) >= top_k:
                break

    return pd.DataFrame(unique_recommendations, columns=['Title', 'Author', 'Genres']).reset_index(drop=True)

In [29]:
query = "The Hunger Games"
recommendations = get_combined_recommendations(query, bx_content, top_k=5)
print(recommendations)

                                               Title              Author  \
0                         Sandman: The Dream Hunters         Neil Gaiman   
1                    A Game of You (Sandman, Book 5)         Neil Gaiman   
2  Battlestar Galactica Classic (Battlestar Galac...     Robert Thurston   
3               Revenant  (Buffy the Vampire Slayer)            Mel Odom   
4  Spike and Dru: Pretty Maids All in a Row (Buff...  Christopher Golden   

                                              Genres  
0  Graphic Novels, Fantasy, Comics, Fiction, Grap...  
1  Comics, Fantasy, Fiction, Graphic Novels, Grap...  
2  Comics, Science Fiction, Graphic Novels Comics...  
3  Buffy The Vampire Slayer, Fantasy, Vampires, Y...  
4  Buffy The Vampire Slayer, Vampires, Fantasy, F...  


In [30]:
query = "The Hunger Games"
recommendations = get_combined_recommendations(query, gbx_content, top_k=5)
print(recommendations)

                                      Title                          Author  \
0  The Walking Dead, Vol. 8: Made To Suffer                  Robert Kirkman   
1                     Sojourn (Dark Elf #3)                  R.A. Salvatore   
2                The Walking Dead. Book Two  Robert Kirkman, Charlie Adlard   
3                               Coming Home               Rosamunde Pilcher   
4                              Annihilation                 Jeff VanderMeer   

                                              Genres  
0  Graphic Novels, Comics, Horror, Zombies, Ficti...  
1  Fantasy, Forgotten Realms, Fiction, High Fanta...  
2  Graphic Novels, Comics, Horror, Zombies, Ficti...  
3  Comics, Graphic Novels, Marvel, Spider Man, Co...  
4  Science Fiction, Fiction, Horror, Fantasy, Mys...  


# Collaborative-filtering Model

##### Memory-based item-similarity CF using kNN

In [31]:
bx_cols_to_drop = ['ISBN', 'title_embeddings', 'combined_embeddings', 'genres_embeddings', 'combined_cluster_label', 'genre_cluster_label']
bx_cf = bx_cleaned.drop(columns=bx_cols_to_drop)
bx_cf

,UserID,User_Rating,Title,Author,Average_Rating,cleaned_title,cleaned_author,Genres,new_combined
0,276733,1,Les Particules Elementaires,Michel Houellebecq,2,le particules elementaires,michel houellebecq,"Fiction, France, Literature, Novels, French Li...",le particules elementaires michel houellebecq ...
1,276746,1,Lightning,Dean R. Koontz,2,lightning,dean r koontz,"Fantasy, Young Adult, Mythology, Fiction, Midd...",lightning dean r koontz fantasy young adult my...
2,276746,1,Manhattan Hunt Club,JOHN SAUL,2,manhattan hunt club,john saul,"Horror, Fiction, Thriller, Mystery, Suspense, ...",manhattan hunt club john saul horror fiction t...
3,276746,1,Dark Paradise,TAMI HOAG,2,dark paradise,tami hoag,"Mystery, Romance, Suspense, Fiction, Romantic ...",dark paradise tami hoag mystery romance suspen...
4,276747,1,How Stella Got Her Groove Back,Terry McMillan,1,stella got groove back,terry mcmillan,"Fiction, Romance, African American, Chick Lit,...",stella got groove back terry mcmillan fiction ...
...,...,...,...,...,...,...,...,...,...
336313,276704,1,Obsidian Butterfly,Laurell K. Hamilton,2,obsidian butterfly,laurell k hamilton,"Urban Fantasy, Vampires, Paranormal, Fantasy, ...",obsidian butterfly laurell k hamilton urban fa...
336314,276704,1,Mirror Image,Sandra Brown,2,mirror image,sandra brown,"Romance, Romantic Suspense, Mystery, Suspense,...",mirror image sandra brown romance romantic sus...
336315,276704,4,Dreamcatcher,Stephen King,2,dreamcatcher,stephen king,"Horror, Fiction, Science Fiction, Thriller, Fa...",dreamcatcher stephen king horror fiction scien...
336316,276704,1,All I Really Need to Know,ROBERT FULGHUM,2,really need know,robert fulghum,"Nonfiction, Humor, Self Help, Philosophy, Essa...",really need know robert fulghum nonfiction hum...


In [32]:
gbx_cols_to_drop = ['id', 'ISBN', 'title_embeddings', 'combined_embeddings', 'genres_embeddings', 'combined_cluster_label', 'genre_cluster_label']
gbx_cf = gbx_cleaned.drop(columns=gbx_cols_to_drop)
gbx_cf

,BookID,Author,Title,Average_Rating,UserID,User_Rating,cleaned_title,cleaned_author,Genres,new_combined
0,2767052,Suzanne Collins,The Hunger Games,4,314,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...
1,2767052,Suzanne Collins,The Hunger Games,4,439,3,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...
2,2767052,Suzanne Collins,The Hunger Games,4,588,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...
3,2767052,Suzanne Collins,The Hunger Games,4,1169,4,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...
4,2767052,Suzanne Collins,The Hunger Games,4,1185,4,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...
...,...,...,...,...,...,...,...,...,...,...
819645,8914,John Keegan,The First World War,4,48281,4,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...
819646,8914,John Keegan,The First World War,4,48386,5,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...
819647,8914,John Keegan,The First World War,4,49007,4,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...
819648,8914,John Keegan,The First World War,4,49383,5,first world war,john keegan,"History, Nonfiction, War, World War I, Militar...",first world war john keegan history nonfiction...


##### Using the 'cleaned_title' column

In [33]:
user_item_matrix_bx = bx_cf.pivot_table(index='UserID', columns='cleaned_title', values='User_Rating').fillna(0)

In [34]:
user_item_matrix_gbx = gbx_cf.pivot_table(index='UserID', columns='cleaned_title', values='User_Rating').fillna(0)

In [35]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
def get_popular_books(df, n=5):
    book_popularity = df.groupby('cleaned_title')['User_Rating'].count().sort_values(ascending=False).head(n)
    popular_books_details = df[df['cleaned_title'].isin(book_popularity.index)][['Title', 'Author', 'Genres']].drop_duplicates().head(n)
    return popular_books_details

In [37]:
def item_similarity_cf_recommendations(book_title, df, n_recommendations=5):
    cleaned_book_title = clean_text(book_title)
    
    # Assuming user_item_matrix is predefined and contains your user-item interaction data
    if 'BookID' in df.columns:
        user_item_matrix = user_item_matrix_gbx
    else:
        user_item_matrix = user_item_matrix_bx
    
    if cleaned_book_title not in user_item_matrix.columns:
        print("Book title not found. Recommending popular books instead.")
        return get_popular_books(df, n_recommendations)
    
    # Initialize and train kNN model
    model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_recommendations+1, n_jobs=-1)
    model_knn.fit(user_item_matrix.T.values)  # Transpose to get item-feature matrix

    # Locate the index of the book title
    book_index = list(user_item_matrix.columns).index(cleaned_book_title)
    distances, indices = model_knn.kneighbors(user_item_matrix.T.iloc[book_index, :].values.reshape(1, -1), n_neighbors=n_recommendations+1)

    # Fetch indices for the recommended books, excluding the book itself
    recommended_indices = indices.flatten()[1:]

    # Fetch book titles for the recommended indices
    recommended_books_titles = [user_item_matrix.columns[i] for i in recommended_indices]
    
    # Fetch the details for the recommended books
    recommended_books_details = df[df['cleaned_title'].isin(recommended_books_titles)].drop_duplicates(subset=['cleaned_title'])
    recommended_books_details = recommended_books_details[['Title', 'Author', 'Genres']].head(n_recommendations)
    
    return recommended_books_details

In [38]:
book_title = "The Hunger Games"
item_similarity_cf_recommendations(book_title, bx_cf, n_recommendations=5)

Book title not found. Recommending popular books instead.


,Title,Author,Genres
72,The Lovely Bones: A Novel,Alice Sebold,"Fiction, Mystery, Young Adult, Contemporary, F..."
82,Wild Animus,Rich Shapero,"Fiction, Fantasy, Novels, Adult Fiction, Audio..."
107,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,Fantasy
295,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,"Fiction, Chick Lit, Contemporary, Adult Fictio..."
368,The Rainmaker,John Grisham,"Fiction, Thriller, Mystery, Legal Thriller, Cr..."


In [39]:
book_title = "The Hunger Games"
item_similarity_cf_recommendations(book_title, gbx_cf, n_recommendations=5)

,Title,Author,Genres
100,Harry Potter and the Philosopher's Stone,"J.K. Rowling, Mary GrandPrÃ©","Fantasy, Fiction, Young Adult, Magic, Children..."
1600,Catching Fire,Suzanne Collins,"Young Adult, Dystopia, Fiction, Fantasy, Scien..."
1900,Mockingjay,Suzanne Collins,"Young Adult, Dystopia, Fiction, Fantasy, Scien..."
2900,The Help,Kathryn Stockett,"Fiction, Historical Fiction, Historical, Adult..."
8500,The Secret Garden,Frances Hodgson Burnett,"Classics, Fiction, Childrens, Young Adult, His..."


In [54]:
book_title = "Twilight"
item_similarity_cf_recommendations(book_title, bx_cf, n_recommendations=5)

,Title,Author,Genres
12321,The Falling Nun: And Other Stories,Pamela Rafael Berkman,"Short Stories, Fiction"
12397,The Book Group Book: A Thoughtful Guide to For...,Ellen Slezak,"Nonfiction, Books About Books"
19017,The Knockout Artist,Harry Crews,"Fiction, Southern, Novels, American, Literatur..."
20119,The It-Doesn'T-Matter Suit,Sylvia Plath,"Fiction, Childrens, Poetry, Short Stories, Pic..."
32219,In Search of Color Everywhere,E. Ethelbert Miller,"Poetry, Anthologies, American"


In [55]:
book_title = "Twilight"
item_similarity_cf_recommendations(book_title, gbx_cf, n_recommendations=5)

,Title,Author,Genres
427057,Midnight,Erin Hunter,"Fiction, Fantasy, Contemporary, Audiobook, Mag..."
447412,Ninth Key,Meg Cabot,"Young Adult, Fantasy, Paranormal, Romance, Gho..."
511684,Moonrise,Erin Hunter,"Fantasy, Young Adult, Paranormal, Vampires, Ro..."
661939,The Sight,Erin Hunter,"Fantasy, Animals, Childrens, Middle Grade, Fic..."
691369,"Starlight (Warriors: The New Prophecy, #4)",Erin Hunter,"Fantasy, Animals, Middle Grade, Fiction, Child..."


##### Using the 'new_combined' column

In [40]:
n_user_item_matrix_bx = bx_cf.pivot_table(index='UserID', columns='new_combined', values='User_Rating').fillna(0)

In [41]:
n_user_item_matrix_gbx = gbx_cf.pivot_table(index='UserID', columns='new_combined', values='User_Rating').fillna(0)

In [42]:
def find_closest_title(book_title, df):
    # Simple string matching for demonstration; consider more sophisticated methods for better accuracy
    cleaned_book_title = clean_text(book_title)
    df['title_similarity'] = df['new_combined'].apply(lambda x: cleaned_book_title in clean_text(x))
    closest_match = df[df['title_similarity']].iloc[0]['new_combined'] if df[df['title_similarity']].shape[0] > 0 else None
    return closest_match

In [43]:
def combined_item_similarity_cf_recommendations(book_title, df, n_recommendations=5):
    combined_query = find_closest_title(book_title, df)
    
    if combined_query is None:
        print("Book title not found. Recommending popular books instead.")
        return get_popular_books(df, n_recommendations)
    
    # Assuming user_item_matrix is predefined and based on 'new_combined'
    if 'BookID' in df.columns:
        user_item_matrix = n_user_item_matrix_gbx
    else:
        user_item_matrix = n_user_item_matrix_bx
    
    if combined_query not in user_item_matrix.columns:
        return get_popular_books(df, n_recommendations)
    
    # Initialize and train kNN model
    model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_recommendations+1, n_jobs=-1)
    model_knn.fit(user_item_matrix.T.values)  # Transpose to get item-feature matrix

    book_index = list(user_item_matrix.columns).index(combined_query)
    distances, indices = model_knn.kneighbors(user_item_matrix.T.iloc[book_index, :].values.reshape(1, -1), n_neighbors=n_recommendations+1)

    recommended_indices = indices.flatten()[1:]
    recommended_books_titles = [user_item_matrix.columns[i] for i in recommended_indices]
    
    recommended_books_details = df[df['new_combined'].isin(recommended_books_titles)].drop_duplicates(subset=['new_combined'])
    recommended_books_details = recommended_books_details[['Title', 'Author', 'Genres']].head(n_recommendations)
    
    return recommended_books_details

In [44]:
book_title = "The Hunger Games"
combined_item_similarity_cf_recommendations(book_title, bx_cf, n_recommendations=5)

Book title not found. Recommending popular books instead.


,Title,Author,Genres
72,The Lovely Bones: A Novel,Alice Sebold,"Fiction, Mystery, Young Adult, Contemporary, F..."
82,Wild Animus,Rich Shapero,"Fiction, Fantasy, Novels, Adult Fiction, Audio..."
107,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,Fantasy
295,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,"Fiction, Chick Lit, Contemporary, Adult Fictio..."
368,The Rainmaker,John Grisham,"Fiction, Thriller, Mystery, Legal Thriller, Cr..."


In [45]:
book_title = "The Hunger Games"
combined_item_similarity_cf_recommendations(book_title, gbx_cf, n_recommendations=5)

,Title,Author,Genres
100,Harry Potter and the Philosopher's Stone,"J.K. Rowling, Mary GrandPrÃ©","Fantasy, Fiction, Young Adult, Magic, Children..."
200,Twilight,Stephenie Meyer,"Young Adult, Romance, Fiction, Vampires, Paran..."
1600,Catching Fire,Suzanne Collins,"Young Adult, Dystopia, Fiction, Fantasy, Scien..."
1900,Mockingjay,Suzanne Collins,"Young Adult, Dystopia, Fiction, Fantasy, Scien..."
2900,The Help,Kathryn Stockett,"Fiction, Historical Fiction, Historical, Adult..."


In [56]:
book_title = "Twilight"
combined_item_similarity_cf_recommendations(book_title, bx_cf, n_recommendations=5)

: 

In [ ]:
book_title = "Twilight"
combined_item_similarity_cf_recommendations(book_title, gbx_cf, n_recommendations=5)

# Hybrid-filtering Model

##### Using BERT embeddings and kNN

In [46]:
bx_cols = ['ISBN', 'combined_cluster_label', 'genre_cluster_label']
bx_collaborative = bx_cleaned.drop(columns=bx_cols)
bx_collaborative.head(5)

,UserID,User_Rating,Title,Author,Average_Rating,cleaned_title,cleaned_author,Genres,new_combined,title_embeddings,combined_embeddings,genres_embeddings
0,276733,1,Les Particules Elementaires,Michel Houellebecq,2,le particules elementaires,michel houellebecq,"Fiction, France, Literature, Novels, French Li...",le particules elementaires michel houellebecq ...,"[-0.22314462, 0.42006478, -0.28728843, -0.1840...","[-0.21087638, 0.38165882, -0.00066206243, -0.3...","[0.11841526, 0.20468897, -0.028159915, -0.2183..."
1,276746,1,Lightning,Dean R. Koontz,2,lightning,dean r koontz,"Fantasy, Young Adult, Mythology, Fiction, Midd...",lightning dean r koontz fantasy young adult my...,"[-0.2500226, 0.1235834, 0.09593897, -0.0904388...","[-0.12596153, 0.2709259, 0.18960203, 0.0491416...","[0.056521952, 0.26575485, 0.12797885, 0.070500..."
2,276746,1,Manhattan Hunt Club,JOHN SAUL,2,manhattan hunt club,john saul,"Horror, Fiction, Thriller, Mystery, Suspense, ...",manhattan hunt club john saul horror fiction t...,"[-0.06912649, -0.19722155, -0.33834106, 0.1077...","[0.33136278, -0.012296129, 0.19435395, 0.10583...","[0.30190024, 0.16326268, 0.5863225, -0.1031865..."
3,276746,1,Dark Paradise,TAMI HOAG,2,dark paradise,tami hoag,"Mystery, Romance, Suspense, Fiction, Romantic ...",dark paradise tami hoag mystery romance suspen...,"[-0.18521202, 0.035213947, -0.043080643, 0.062...","[0.14718063, 0.0041864626, 0.14003608, 0.03475...","[0.32722577, 0.1615243, 0.592917, -0.18396236,..."
4,276747,1,How Stella Got Her Groove Back,Terry McMillan,1,stella got groove back,terry mcmillan,"Fiction, Romance, African American, Chick Lit,...",stella got groove back terry mcmillan fiction ...,"[0.021076923, -0.36269993, 0.056049906, 0.0045...","[-0.10719866, -0.010877345, 0.28632286, 0.0426...","[-0.06881369, 0.007822593, 0.1244007, -0.25172..."


In [47]:
gbx_cols = ['id', 'ISBN', 'combined_cluster_label', 'genre_cluster_label']
gbx_collaborative = gbx_cleaned.drop(columns=bx_cols)
gbx_collaborative.head(5)

,id,BookID,Author,Title,Average_Rating,UserID,User_Rating,cleaned_title,cleaned_author,Genres,new_combined,title_embeddings,combined_embeddings,genres_embeddings
0,1,2767052,Suzanne Collins,The Hunger Games,4,314,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...","[-0.03419616, 0.14870769, 0.5226705, -0.080204..."
1,1,2767052,Suzanne Collins,The Hunger Games,4,439,3,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...","[-0.03419616, 0.14870769, 0.5226705, -0.080204..."
2,1,2767052,Suzanne Collins,The Hunger Games,4,588,5,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...","[-0.03419616, 0.14870769, 0.5226705, -0.080204..."
3,1,2767052,Suzanne Collins,The Hunger Games,4,1169,4,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...","[-0.03419616, 0.14870769, 0.5226705, -0.080204..."
4,1,2767052,Suzanne Collins,The Hunger Games,4,1185,4,hunger game,suzanne collins,"Young Adult, Fiction, Fantasy, Dystopia, Scien...",hunger game suzanne collins young adult fictio...,"[-0.35120225, -0.6337399, -0.2950579, -0.18312...","[-0.10558018, 0.06420389, 0.4107097, 0.1490544...","[-0.03419616, 0.14870769, 0.5226705, -0.080204..."


In [48]:
item_similarity_matrix_bx = cosine_similarity(bx_title_embedding)

In [49]:
item_similarity_matrix_gbx = cosine_similarity(gbx_title_embedding)

In [50]:
def hybrid_bert_recommendations(book_title, df, n_recommendations=5):
    # Construct item-item similarity matrix
    if 'BookID' in df.columns:
        item_similarity_matrix = item_similarity_matrix_gbx
    else:
        item_similarity_matrix = item_similarity_matrix_bx
    # item_similarity_matrix = construct_item_similarity_matrix(df)
    
    # Find the index of the book title in the dataframe
    book_index = df[df['Title'] == book_title].index[0]
    
    # Use kNN with item similarity matrix to find similar items
    model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=n_recommendations, n_jobs=-1)
    model_knn.fit(item_similarity_matrix)
    
    # Find nearest neighbors (similar items) for the specified book title
    distances, indices = model_knn.kneighbors([item_similarity_matrix[book_index]], n_neighbors=n_recommendations+1)
    
    # Exclude the first item (itself) and get top n recommendations
    recommended_indices = indices.flatten()[1:]
    recommended_books_details = df.iloc[recommended_indices][['Title', 'Author', 'Genres']]
    
    return recommended_books_details

In [53]:
book_title = "The Hunger Games"
hybrid_bert_recommendations(book_title, gbx_cf, n_recommendations=5)

,Title,Author,Genres
516,The Fault in Our Stars,John Green,"Young Adult, Fiction, Contemporary, Realistic ..."
585,The Fault in Our Stars,John Green,"Young Adult, Fiction, Contemporary, Realistic ..."
1006,The Kite Runner,Khaled Hosseini,"Fiction, Historical Fiction, Contemporary, Nov..."
7257,The Devil Wears Prada,Lauren Weisberger,"Chick Lit, Fiction, Contemporary, Romance, Adu..."
904,Pride and Prejudice,Jane Austen,"Classics, Fiction, Romance, Historical Fiction..."


# Hybrid-filtering Model

#### Using Weighted Hybrid / Mixed Hybrid approach

In [ ]:
# bx_content OR gbx_content
# bx_cf OR gbx_cf

In [ ]:
def hybrid_recommendations(book_title, df, n_recommendations=5):
    # Step 1: Generate content-based recommendations
    content_based_recommendations = get_combined_recommendations(book_title, df, top_k=n_recommendations*2)
    
    # Step 2: Generate CF recommendations
    cf_recommendations = item_similarity_cf_recommendations(book_title, df, n_recommendations=n_recommendations*2)
    
    # Step 3: Merge and filter recommendations to ensure uniqueness and exclude the query title
    combined_recommendations = pd.concat([content_based_recommendations, cf_recommendations]).drop_duplicates(subset=['Title', 'Author']).reset_index(drop=True)
    
    # Exclude the query title
    combined_recommendations = combined_recommendations[combined_recommendations['Title'].str.lower() != book_title.lower()]
    
    # Limit to the requested number of recommendations
    if len(combined_recommendations) > n_recommendations:
        combined_recommendations = combined_recommendations.head(n_recommendations)
    
    return combined_recommendations
